<a href="https://colab.research.google.com/github/tarini-py/Machine-Learning/blob/main/1_cars24__linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Temp-LR-1-D71


In [ ]:
import numpy as np

In [ ]:
x = [[1.0, 0.0], [1.0, 1.0], [1.0, 2.0], [1.0, 3.0], [1.0, 4.0], [1.0, 5.0], [1.0, 6.0], [1.0, 7.0], [1.0, 8.0], [1.0, 9.0]]
y = [32.69, 30.56, 32.04, 32.27, 35.98, 34.39, 40.25, 39.24, 37.14, 39.38]
w = [30.0, 1.0]

In [ ]:
x = np.asarray(x)
w = np.asarray(w)
y = np.asarray(y)

In [ ]:
x,y,w

(array([[1., 0.],
        [1., 1.],
        [1., 2.],
        [1., 3.],
        [1., 4.],
        [1., 5.],
        [1., 6.],
        [1., 7.],
        [1., 8.],
        [1., 9.]]),
 array([32.69, 30.56, 32.04, 32.27, 35.98, 34.39, 40.25, 39.24, 37.14,
        39.38]),
 array([30.,  1.]))

In [ ]:
x.shape, y.shape, w.shape

((10, 2), (10,), (2,))

In [ ]:
w.reshape((1,2))

array([[30.,  1.]])

In [ ]:
y-np.dot(x,w.reshape((1,2)).T).reshape(-1)

array([ 2.69, -0.44,  0.04, -0.73,  1.98, -0.61,  4.25,  2.24, -0.86,
        0.38])

In [ ]:
sum((y-np.dot(x,w.reshape((1,2)).T).reshape(-1))**2)/(2*10)

np.float64(1.811039999999999)

In [ ]:
w.reshape(1,-1)

array([[30.,  1.]])

#--

In [ ]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 3.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, OrdinalEncoder
from category_encoders.target_encoder import TargetEncoder
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, root_mean_squared_error

In [ ]:
!gdown 15tIjPrstV2RnOMv2VSIKW7yW_M3KjyNa

Downloading...
From: https://drive.google.com/uc?id=15tIjPrstV2RnOMv2VSIKW7yW_M3KjyNa
To: /content/cars24-car-price.csv
100% 1.79M/1.79M [00:00<00:00, 82.2MB/s]


In [ ]:
df = pd.read_csv('/content/cars24-car-price.csv')
df.head()

,full_name,selling_price,year,seller_type,km_driven,fuel_type,transmission_type,mileage,engine,max_power,seats
0,Maruti Alto Std,1.20,2012.0,Individual,120000,Petrol,Manual,19.70,796.0,46.30,5.0
1,Hyundai Grand i10 Asta,5.50,2016.0,Individual,20000,Petrol,Manual,18.90,1197.0,82.00,5.0
2,Hyundai i20 Asta,2.15,2010.0,Individual,60000,Petrol,Manual,17.00,1197.0,80.00,5.0
3,Maruti Alto K10 2010-2014 VXI,2.26,2012.0,Individual,37000,Petrol,Manual,20.92,998.0,67.10,5.0
4,Ford Ecosport 2015-2021 1.5 TDCi Titanium BSIV,5.70,2015.0,Dealer,30000,Diesel,Manual,22.77,1498.0,98.59,5.0


In [ ]:
df_org = df.copy(deep = True)
np.shares_memory(df, df_org)

False

In [ ]:
df.shape

(19980, 11)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19980 entries, 0 to 19979
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   full_name          19980 non-null  object 
 1   selling_price      19980 non-null  float64
 2   year               19980 non-null  float64
 3   seller_type        19980 non-null  object 
 4   km_driven          19980 non-null  int64  
 5   fuel_type          19980 non-null  object 
 6   transmission_type  19980 non-null  object 
 7   mileage            19980 non-null  float64
 8   engine             19980 non-null  float64
 9   max_power          19980 non-null  float64
 10  seats              19980 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.7+ MB


In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
selling_price,19980.0,7.392066,9.103088,0.25,3.40,5.20,7.85,395.0
year,19980.0,2014.525125,3.249185,1991.00,2013.00,2015.00,2017.00,2021.0
km_driven,19980.0,58244.878979,51725.090445,100.00,31164.25,52000.00,74000.00,3800000.0
mileage,19980.0,19.347219,4.620053,0.00,16.80,19.16,22.32,120.0
engine,19980.0,1476.327401,520.449398,0.00,1197.00,1248.00,1582.00,6752.0
max_power,19980.0,97.854443,45.080670,5.00,73.90,86.70,112.00,626.0
seats,19980.0,5.317868,0.835370,2.00,5.00,5.00,5.00,14.0


#Feature engineering

In [ ]:
df['full_name'].head(3)

,full_name
0,Maruti Alto Std
1,Hyundai Grand i10 Asta
2,Hyundai i20 Asta


In [ ]:
df['make'] = df['full_name'].str.split(' ').str[0]
df['model'] = df['full_name'].str.split(' ').str[1:].str.join(' ')

In [ ]:
df.drop(columns = ['full_name'], inplace = True)

In [ ]:
df.shape

(19980, 12)

In [ ]:
cat_cols = df.select_dtypes(include = ['object','category']).columns
cat_cols

Index(['seller_type', 'fuel_type', 'transmission_type', 'make', 'model'], dtype='object')

In [ ]:
num_cols = df.select_dtypes(include = ['int64','float64']).columns
num_cols

Index(['selling_price', 'year', 'km_driven', 'mileage', 'engine', 'max_power',
       'seats'],
      dtype='object')

In [ ]:
for i in cat_cols:
  print(i,' - ', df[i].nunique())
  display(df[i].value_counts().head(3))
  print("======")

seller_type  -  3


,count
seller_type,
Dealer,11973
Individual,7817
Trustmark Dealer,190


fuel_type  -  5


,count
fuel_type,
Diesel,9817
Petrol,9767
CNG,316


transmission_type  -  2


,count
transmission_type,
Manual,16029
Automatic,3951


make  -  42


,count
make,
Maruti,5650
Hyundai,3646
Honda,1781


model  -  3321


,count
model,
Swift Dzire VDI,210
Alto 800 LXI,189
Wagon R VXI,164


All categorical features except 'make' and 'model' will get OneHotEncoded

'make' and 'model' will get TargetEncoded(here target is selling_price), so this will encoded after tarin_test_split to avoid data leakage

In [ ]:
df_org['seats'].value_counts()

,count
seats,
5.0,16681
7.0,2370
8.0,440
4.0,225
6.0,143
9.0,72
10.0,26
2.0,21
14.0,2


In [ ]:
def merge_seats(x):
  if x==2:
      return '2'
  elif x == 4:
      return '4'
  elif x == 5 or x == 6:
      return '5-6'
  elif x == 7 or x==8:
      return '7-8'
  else:
      return '9+'


df['seats'] = df['seats'].apply(merge_seats)
df.head()

,selling_price,year,seller_type,km_driven,fuel_type,transmission_type,mileage,engine,max_power,seats,make,model
0,1.20,2012.0,Individual,120000,Petrol,Manual,19.70,796.0,46.30,5-6,Maruti,Alto Std
1,5.50,2016.0,Individual,20000,Petrol,Manual,18.90,1197.0,82.00,5-6,Hyundai,Grand i10 Asta
2,2.15,2010.0,Individual,60000,Petrol,Manual,17.00,1197.0,80.00,5-6,Hyundai,i20 Asta
3,2.26,2012.0,Individual,37000,Petrol,Manual,20.92,998.0,67.10,5-6,Maruti,Alto K10 2010-2014 VXI
4,5.70,2015.0,Dealer,30000,Diesel,Manual,22.77,1498.0,98.59,5-6,Ford,Ecosport 2015-2021 1.5 TDCi Titanium BSIV


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19980 entries, 0 to 19979
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   selling_price      19980 non-null  float64
 1   year               19980 non-null  float64
 2   seller_type        19980 non-null  object 
 3   km_driven          19980 non-null  int64  
 4   fuel_type          19980 non-null  object 
 5   transmission_type  19980 non-null  object 
 6   mileage            19980 non-null  float64
 7   engine             19980 non-null  float64
 8   max_power          19980 non-null  float64
 9   seats              19980 non-null  object 
 10  make               19980 non-null  object 
 11  model              19980 non-null  object 
dtypes: float64(5), int64(1), object(6)
memory usage: 1.8+ MB


#Train-test-split

In [ ]:
X = df.drop(columns = ['selling_price']) #Features
y = df['selling_price'] #Target

In [ ]:
X.shape, y.shape

((19980, 11), (19980,))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((13986, 11), (13986,), (5994, 11), (5994,))

In [ ]:
X_train.head(3)

,year,seller_type,km_driven,fuel_type,transmission_type,mileage,engine,max_power,seats,make,model
3493,2008.0,Dealer,57000,Petrol,Automatic,13.90,1799.0,130.20,5-6,Honda,Civic 1.8 V AT
17862,2018.0,Dealer,38000,Diesel,Manual,22.54,1396.0,88.76,5-6,Hyundai,i20 Asta Option Diesel
1459,2013.0,Dealer,29000,Diesel,Automatic,18.25,1968.0,187.74,5-6,Audi,A4 35 TDI Premium Plus


In [ ]:
y_train

,selling_price
3493,2.50
17862,8.75
1459,12.50
9780,2.90
19816,2.75
...,...
11284,2.45
11964,4.25
5390,6.21
860,5.10


#Column Transformer & Pipelines


In [ ]:
cat_cols = df.select_dtypes(include = ['object','category']).columns
num_cols = df.select_dtypes(include = ['int64','float64']).columns

In [ ]:
cat_cols, num_cols

(Index(['seller_type', 'fuel_type', 'transmission_type', 'seats', 'make',
        'model'],
       dtype='object'),
 Index(['selling_price', 'year', 'km_driven', 'mileage', 'engine', 'max_power'], dtype='object'))

In [ ]:
ohe_cat_cols = cat_cols.drop(['make','model','seats'])
ohe_cat_cols

Index(['seller_type', 'fuel_type', 'transmission_type'], dtype='object')

In [ ]:
lb_cat_cols = ['seats']
lb_cat_cols

['seats']

In [ ]:
te_cat_cols = ['make','model']
te_cat_cols

['make', 'model']

In [ ]:
train_num_cols = num_cols.drop(['selling_price'])
train_num_cols

Index(['year', 'km_driven', 'mileage', 'engine', 'max_power'], dtype='object')

In [ ]:
['2', '4', '5-6', '7-8', '9+'][::-1]

['9+', '7-8', '5-6', '4', '2']

In [ ]:
ord_enc = OrdinalEncoder(
    categories = [
        ['9+', '7-8', '5-6', '4', '2']#seats
      ],
    handle_unknown = 'use_encoded_value',
    unknown_value = -1
)

In [ ]:
preprocessor = ColumnTransformer(
    transformers = [
                     ('cat', OneHotEncoder(drop='first', handle_unknown = 'ignore'), ohe_cat_cols),
                     ('ord', ord_enc, lb_cat_cols),
                     ('te', TargetEncoder(), te_cat_cols),
                     ('num', MinMaxScaler(), train_num_cols)
    ]
)

ColumnTransformer only transform or scale encode X_train
<br> to scale y_train we use TarnsformedTargetRegressor

In [ ]:
model = LinearRegression()

reg = TransformedTargetRegressor(
    regressor = model,
    transformer = MinMaxScaler()
)

In [ ]:
pipeline = Pipeline(
    steps = [
             ('preprocess', preprocessor),
             ('reg', reg)
    ]
)

In [ ]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  Index(['seller_type', 'fuel_type', 'transmission_type'], dtype='object')),
                                                 ('ord',
                                                  OrdinalEncoder(categories=[['9+',
                                                                              '7-8',
                                                                              '5-6',
                                                                              '4',
                                                                              '2']],
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['seats']),
                                                 ('te', TargetEncoder(),
                                                  ['make', 'model']),
                                                 ('num', MinMaxScaler(),
                                                  Index(['year', 'km_driven', 'mileage', 'engine', 'max_power'], dtype='object'))])),
                ('reg',
                 TransformedTargetRegressor(regressor=LinearRegression(),
                                            transformer=MinMaxScaler()))])

In [ ]:
y_pred = pipeline.predict(X_test)
y_pred

array([-1.1777893 ,  8.79638632,  6.8800504 , ...,  6.12542849,
        5.54262111, 11.75676017])

In [ ]:
pd.DataFrame({'y_test':y_test, 'y_pred':y_pred})

,y_test,y_pred
4013,2.70,-1.177789
2085,9.51,8.796386
4771,7.50,6.880050
10631,19.00,17.346435
15440,12.75,20.833991
...,...,...
9243,2.65,-2.728598
6934,3.60,4.871402
18241,6.00,6.125428
15367,6.60,5.542621


In [ ]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)

mse, r2, mae, rmse

(22.664511411858058, 0.6611810416775176, 2.7219382184442993, 4.760725933285602)

In [ ]:
pipeline.score(X_train, y_train)

0.7094095849696846

In [ ]:
pipeline.score(X_test, y_test)

0.6611810416775176

In [ ]:
pipeline.named_steps['reg'].regressor_.coef_

array([ 0.00054071, -0.00119363, -0.00424666,  0.00047689,  0.00031495,
       -0.0008904 ,  0.00258345,  0.00428545,  0.00093392,  0.00404891,
        0.02944425, -0.03536973,  0.02573929,  0.03624926,  0.08528937])

In [ ]:
pipeline.named_steps['reg'].regressor_.intercept_

np.float64(-0.07087859328980142)

In [ ]:
for i,j in zip(preprocessor.get_feature_names_out(),pipeline.named_steps['reg'].regressor_.coef_):
  print(i,' : ',j)

cat__seller_type_Individual  :  0.000540710519783637
cat__seller_type_Trustmark Dealer  :  -0.0011936250214444944
cat__fuel_type_Diesel  :  -0.0042466557972332475
cat__fuel_type_Electric  :  0.00047688835401747296
cat__fuel_type_LPG  :  0.0003149492452205502
cat__fuel_type_Petrol  :  -0.0008903997819939216
cat__transmission_type_Manual  :  0.0025834500219239784
ord__seats  :  0.0042854502160409366
te__make  :  0.0009339174940503829
te__model  :  0.0040489145778088214
num__year  :  0.029444245383483104
num__km_driven  :  -0.03536973193486662
num__mileage  :  0.02573929277039836
num__engine  :  0.036249263219949396
num__max_power  :  0.08528936898722664


In [ ]:
X_train[['year','mileage','engine']]

,year,mileage,engine
3493,2008.0,13.90,1799.0
17862,2018.0,22.54,1396.0
1459,2013.0,18.25,1968.0
9780,2010.0,21.10,1197.0
19816,2012.0,21.12,1248.0
...,...,...,...
11284,2009.0,17.50,1298.0
11964,2017.0,24.00,1120.0
5390,2016.0,18.60,1197.0
860,2016.0,27.39,1248.0


In [ ]:
X_train[['year','mileage','engine']]

,year,mileage,engine
3493,2007.0,12.90,1798.0
17862,2017.0,21.54,1395.0
1459,2012.0,17.25,1967.0
9780,2009.0,20.10,1196.0
19816,2011.0,20.12,1247.0
...,...,...,...
11284,2008.0,16.50,1297.0
11964,2016.0,23.00,1119.0
5390,2015.0,17.60,1196.0
860,2015.0,26.39,1247.0


In [ ]:
y_train

,selling_price
3493,2.50
17862,8.75
1459,12.50
9780,2.90
19816,2.75
...,...
11284,2.45
11964,4.25
5390,6.21
860,5.10


In [ ]:
y_hat = np.ones(13986,)
y_hat

array([1., 1., 1., ..., 1., 1., 1.])

In [ ]:
y_train - 1

,selling_price
3493,1.50
17862,7.75
1459,11.50
9780,1.90
19816,1.75
...,...
11284,1.45
11964,3.25
5390,5.21
860,4.10


#Implementing LR from scartch

In [ ]:
import numpy as np
import pandas as pd
class linear_regression:

    '''
    >> model = linera_regression()
    >> model.fit(X_train, y_train)
    >> model.preict(X_test)
    '''
    def __init__(self, learning_rate = 0.1, iteration = 100):

        self.learning_rate = learning_rate
        self.iteration = iteration


    def fit(self, X_train, y_train):

        self.X_train = X_train
        self.y_train = y_train

        #self.m -> no. of training samples( rows )
        #self.d -> no. of features( cols )
        self.m, self.d = X_train.shape

        #initializing Weights(self.W) and bias(self.b) with 0 to start with
        self.W = np.zeros(self.d)
        self.b = 0

        #list of error to keep track of errors
        self.errors = []

        for i in range(self.iteration):

            #calculating y_hat(prediction) based on initial W, b
            y_hat = self.predict(self.X_train)

            #calculating partial derivatives
            dW = (-1) * np.dot(self.X_train.T, (self.y_train - y_hat))/self.m
            db = (-1) * np.sum((self.y_train - y_hat))/self.m

            #updating weights & bias using the above calculated derivatives
            self.W -= self.learning_rate * dW
            self.b -= self.learning_rate * db

            error = np.square( y_train - self.predict(y_train) )
            self.errors.append(error)
            if error == 0:
                break

    def predict(self, X):
        return np.dot(X, self.W) + self.b

